In [1]:
# import the librarys 
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


In [2]:
# import the data from comma seprated values (csv file)
trainDf=pd.read_csv("train.csv")
testDf=pd.read_csv("test.csv")
print(trainDf.shape)
print(testDf.shape)

(40000, 15)
(10000, 14)


In [3]:
trainDf.columns

Index(['id', 'song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence',
       'song_popularity'],
      dtype='object')

In [4]:
testDf.columns

Index(['id', 'song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence'],
      dtype='object')

In [3]:
# first i drop Id column form data  because i don't think it will help me for analysis 
trainDf.drop('id',axis=1,inplace=True)
testDf.drop('id',axis=1,inplace=True)



In [6]:
trainDf.sample(5)

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,song_popularity
8682,217594.0,0.062394,0.768167,0.765830,0.001428,6.0,0.114057,-3.891405,1,0.080001,148.983298,3,0.779235,1
22380,172987.0,0.014824,0.616125,0.818873,-0.000208,0.0,0.087002,-4.402016,0,0.253875,151.395277,3,0.658068,1
37596,209337.0,0.647925,0.471704,0.791125,0.893206,7.0,0.751185,-4.618551,0,0.035911,164.059659,3,0.317951,1
30265,210606.0,0.086558,0.708314,0.922790,0.001898,10.0,0.117139,NaN,0,0.145635,118.926432,4,0.697577,1
26144,209125.0,0.020585,0.621575,0.696875,NaN,5.0,0.153165,-4.921654,0,0.129106,106.742086,4,0.376641,0


In [7]:
testDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_duration_ms  9043 non-null   float64
 1   acousticness      9050 non-null   float64
 2   danceability      9024 non-null   float64
 3   energy            8945 non-null   float64
 4   instrumentalness  9000 non-null   float64
 5   key               9008 non-null   float64
 6   liveness          8994 non-null   float64
 7   loudness          8974 non-null   float64
 8   audio_mode        10000 non-null  int64  
 9   speechiness       10000 non-null  float64
 10  tempo             10000 non-null  float64
 11  time_signature    10000 non-null  int64  
 12  audio_valence     10000 non-null  float64
dtypes: float64(11), int64(2)
memory usage: 1015.8 KB


In [8]:
trainDf.describe()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,song_popularity
count,35899.000000,36008.000000,35974.000000,36025.000000,36015.000000,35935.000000,35914.000000,36043.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,193165.847572,0.276404,0.570951,0.683932,0.036527,5.042605,0.198514,-7.407596,0.321150,0.094107,116.562815,3.394375,0.580645,0.364400
std,45822.127679,0.297928,0.190010,0.212662,0.150024,3.372728,0.151670,3.877198,0.466924,0.083591,26.167911,0.524405,0.237351,0.481268
min,25658.000000,-0.013551,0.043961,-0.001682,-0.004398,0.000000,0.027843,-32.117911,0.000000,0.015065,62.055779,2.000000,0.013398,0.000000
25%,166254.500000,0.039618,0.424760,0.539276,0.000941,2.000000,0.111796,-9.578139,0.000000,0.038500,96.995309,3.000000,0.398669,0.000000
50%,186660.000000,0.140532,0.608234,0.704453,0.001974,5.000000,0.135945,-6.345413,0.000000,0.055881,113.795959,3.000000,0.598827,0.000000
75%,215116.000000,0.482499,0.718464,0.870503,0.003225,8.000000,0.212842,-4.620711,1.000000,0.118842,128.517383,4.000000,0.759635,1.000000
max,491671.000000,1.065284,0.957131,1.039741,1.075415,11.000000,1.065298,-0.877346,1.000000,0.560748,219.163578,5.000000,1.022558,1.000000


In [4]:
# Step 1 -> train/test/split
X_train,X_test,y_train,y_test = train_test_split(trainDf.drop(columns=["song_popularity"]),
                                                 trainDf["song_popularity"],
                                                 test_size=0.2,
                                                random_state=42)

In [5]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(32000, 13)
(8000, 13)
(32000,)
(8000,)


In [11]:
y_test

32823    1
16298    0
28505    0
6689     1
26893    0
        ..
13618    0
35165    1
16359    1
3842     1
37682    0
Name: song_popularity, Length: 8000, dtype: int64

# Pipe Line

In [19]:
num_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy="median")),
    
    ('std_sacaler',StandardScaler())])

In [20]:
X_train_tr=num_pipeline.fit_transform(X_train)
X_test_tr=num_pipeline.fit_transform(X_test)
testDf_tr=num_pipeline.fit_transform(testDf)

In [21]:
X_train=pd.DataFrame(data=X_train_tr,columns=X_train.columns)
X_test=pd.DataFrame(data=X_test_tr,columns=X_test.columns)
testDf=pd.DataFrame(data=testDf_tr,columns=testDf.columns)

In [118]:
from sklearn import set_config
set_config(display='diagram')

In [119]:
num_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('std_sacaler', StandardScaler())])

In [121]:
num_pipeline.steps

[('imputer', SimpleImputer(strategy='median')),
 ('std_sacaler', StandardScaler())]

# outlier detection and removel

In [22]:
def outliers_detection(data):
    columnsForOutlierDetection=list(data.columns)
    for column in columnsForOutlierDetection:
        
        Q1 = np.percentile(data[column], 25)
        Q3 = np.percentile(data[column], 75)
        IQR = Q3-Q1

        UB = Q3 + 1.5*IQR
        LB = Q1 - 1.5*IQR

        # Upper bound operation
        data[column] = np.where(data[column] > UB, UB, data[column])

        # Lower bound operation
        data[column] = np.where(data[column] < LB, LB, data[column])
    
    return data

In [23]:
testDf=outliers_detection(testDf)
X_test=outliers_detection(X_test)
X_train=outliers_detection(X_train)

In [117]:
X_train.describe()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
count,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,3.200000e+04,32000.000000,32000.000000,3.200000e+04,3.200000e+04,3.200000e+04,3.200000e+04,3.200000e+04
mean,-0.024433,-0.000082,0.000002,0.000631,-0.215929,4.477738e-16,-0.170584,0.034664,1.948719e-16,2.433106e-16,4.939671e-16,1.564970e-15,2.320678e-16
std,0.878260,0.999798,1.000011,0.998147,0.012343,1.000016e+00,0.542291,0.887910,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00
min,-1.996991,-0.966213,-2.882331,-2.830092,-0.244807,-1.571361e+00,-1.132727,-2.238729,-6.824995e-01,-1.065208e+00,-2.121938e+00,-2.653718e+00,-2.392131e+00
25%,-0.550130,-0.752485,-0.711609,-0.617466,-0.223934,-9.463829e-01,-0.537737,-0.486642,-6.824995e-01,-7.261645e-01,-7.575445e-01,-7.482524e-01,-7.662357e-01
50%,-0.132163,-0.428893,0.181338,0.095348,-0.217640,-8.915703e-03,-0.387757,0.254000,-6.824995e-01,-4.745878e-01,-1.008548e-01,-7.482524e-01,7.408929e-02
75%,0.414445,0.591596,0.735539,0.857618,-0.210019,9.285515e-01,0.057416,0.681415,1.465203e+00,4.314927e-01,4.755557e-01,1.157213e+00,7.519131e-01
max,1.861306,2.607716,2.109741,1.754097,-0.189146,1.866019e+00,0.950146,1.746063,1.465203e+00,2.167978e+00,2.325206e+00,3.062678e+00,1.862182e+00


In [24]:
y_train.value_counts()

0    20335
1    11665
Name: song_popularity, dtype: int64

## Model building

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree  import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score,f1_score,recall_score,accuracy_score,confusion_matrix


In [38]:
def Model_Result(actual,pred):
    print("Confusion Matrix")
    confusion=confusion_matrix(actual,pred)
    print(confusion)
    print("**********************************************************")
    print('Accuracy -',accuracy_score(actual,pred))
    print("Recall -",recall_score(actual,pred))
    print("Precision -",precision_score(actual,pred))
    print("F1 Score -",f1_score(actual,pred))
    
    Mat= [accuracy_score(actual,pred),recall_score(actual,pred),precision_score(actual,pred),f1_score(actual,pred)]
    return Mat

In [44]:
validation_dict={}
def different_model(model):
    ml=model
    
    ml.fit(X_train,y_train)
    
    pred_y = ml.predict(X_test)
    
    return_list=Model_Result(y_test,pred_y)
    validation_dict[model]=return_list

In [48]:
# Logistics regression
different_model(LogisticRegression(random_state=2))

Confusion Matrix
[[5088    1]
 [2909    2]]
**********************************************************
Accuracy - 0.63625
Recall - 0.0006870491240123669
Precision - 0.6666666666666666
F1 Score - 0.0013726835964310226


In [34]:
# DecisionTreeClassifier
different_model(DecisionTreeClassifier())

Confusion Matrix
[[3222 1867]
 [1832 1079]]
**********************************************************
Accuracy - 0.537625
Recall - 0.3706630024046719
Precision - 0.3662593346911066
F1 Score - 0.36844801092709584


In [49]:
# RandomForestClassifier
different_model(RandomForestClassifier(random_state=1))

Confusion Matrix
[[4824  265]
 [2680  231]]
**********************************************************
Accuracy - 0.631875
Recall - 0.07935417382342838
Precision - 0.4657258064516129
F1 Score - 0.1356031699442325


In [36]:
# KNeighborsClassifier
different_model(KNeighborsClassifier())

Confusion Matrix
[[3755 1334]
 [2131  780]]
**********************************************************
Accuracy - 0.566875
Recall - 0.2679491583648231
Precision - 0.36896877956480606
F1 Score - 0.31044776119402984


In [37]:
# SVC
different_model(SVC())

Confusion Matrix
[[5088    1]
 [2910    1]]
**********************************************************
Accuracy - 0.636125
Recall - 0.00034352456200618345
Precision - 0.5
F1 Score - 0.0006865774116031583


## hyperparameter tuning

In [ ]:
# know i select three algorithm for hyperprameter tuning because they three give me best result
# 1-SVC
# 2- RandomForestClassifier 
# 3- Logistics regression

In [51]:
# decsion tree gridsearchcv
from sklearn.model_selection import GridSearchCV

In [61]:
# parameter for random forest 
my_pram_grid={"n_estimators":[50,100],
              "max_features":["sqrt","log2",' auto'],
              "min_samples_split":[300,500,200,None]}

In [89]:
# parameter for logistic regression 
my_pram_grid1={"solver":['lbfgs','liblinear','sag','saga'],
              "penalty":['l1','l2',None],
              "C":[100,10,1,0.1]}


In [90]:
# parameter for  SVC
my_pram_grid2 = {'C': [0.1, 1, 10], 
              'gamma': [1, 0.01],
              'kernel': ['rbf']} 

In [95]:
def hyperparameter(model,my_pram):
    grid_search_model=GridSearchCV(estimator=model,param_grid=my_pram,cv=5,n_jobs=-1)
    grid = grid_search_model.fit(X_train,y_train)
    print('accuracy',grid.best_score_) 
    print('parameter',grid.best_params_) 

In [96]:
hyperparameter(LogisticRegression(),my_pram_grid1)

accuracy 0.63540625
parameter {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}


In [97]:
hyperparameter(RandomForestClassifier(),my_pram_grid)

accuracy 0.6355625
parameter {'max_features': 'sqrt', 'min_samples_split': 300, 'n_estimators': 100}


In [120]:
#hyperparameter(SVC(),my_pram_grid2)

## final model

In [111]:
# random forest 
Model=RandomForestClassifier(random_state=123,max_features='sqrt',min_samples_split=300,n_estimators=100)
m=Model.fit(X_train,y_train)

In [114]:
Test_Pred=m.predict(X_test)

In [115]:
# Validation on Testset
from sklearn.metrics import classification_report 
print(classification_report(y_test, Test_Pred))

              precision    recall  f1-score   support

           0       0.64      1.00      0.78      5089
           1       0.65      0.00      0.01      2911

    accuracy                           0.64      8000
   macro avg       0.64      0.50      0.39      8000
weighted avg       0.64      0.64      0.50      8000

